In [2]:
import numpy as np
import pandas as pd
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
import xgboost
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from time import time
from sklearn.svm import SVC, LinearSVC
from fast_ml.model_development import train_valid_test_split
from scipy.sparse import csr_matrix

In [3]:
data=pd.read_csv("./Desktop/nlp_a1/train.csv")

In [4]:
data1=pd.read_csv("./Desktop/nlp_a1/train.csv")

In [5]:
punctuation = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
Stopwords = set(stopwords.words('english'))

In [6]:
def preprocess1(txt):
    txt = txt.lower() 
    txt = punctuation.sub(' ', txt)
    text = BAD_SYMBOLS_RE.sub('', txt) 
    txt = ' '.join(word for word in txt.split() if word not in Stopwords) # delete stopwors from text
    return txt

def preprocess2(txt):
    txt = txt.lower() 
    txt = punctuation.sub(' ', txt)
    text = BAD_SYMBOLS_RE.sub('', txt) 
    return txt

In [7]:
data['profile']=data['profile'].apply(preprocess1)
# data1['profile']=data1['profile'].apply(preprocess2)

In [ ]:
data['profession'].value_counts()
data['profile'].apply(lambda x: len(x.split(' '))).sum()
data['profession'].values

In [ ]:
data_train=pd.DataFrame(columns=['profile','profession'])
data_val=pd.DataFrame(columns=['profile','profession'])
data_test=pd.DataFrame(columns=['profile','profession'])

data=data.sample(frac=1, random_state=143).reset_index()

prof=data['profession'].unique()
a=0
for i in prof:
    df=data[data['profession']==i].reset_index().drop(columns=['index'])
    rows=int(df.shape[0]/10)
    f=df.shape[0]-rows*10
    l1=[i for i in range(0,rows*8+f)]
    l2=[i for i in range(rows*8+f,rows*9+f)]
    l3=[i for i in range(rows*9+f,df.shape[0])]
    df1=df.filter(items=l1,axis=0)
    df2=df.filter(items=l2,axis=0)
    df3=df.filter(items=l3,axis=0)
    data_train=pd.concat([data_train,df1], ignore_index=True)
    data_val=pd.concat([data_val,df2], ignore_index=True)
    data_test=pd.concat([data_test,df3], ignore_index=True)
data_train=data_train.sample(frac=1, random_state=143).reset_index()
data_val=data_val.sample(frac=1, random_state=143).reset_index()
data_test=data_test.sample(frac=1, random_state=143).reset_index()

In [ ]:
X_train,y_train=data_train.profile,data_train.profession
X_val,y_val=data_val.profile,data_val.profession
X_test,y_test=data_test.profile,data_test.profession

In [8]:
X = data.profile
y = data.profession
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 1667)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state = 1667)

In [ ]:
# Naive Bayes Classifier for Multinomial Models

from sklearn.naive_bayes import MultinomialNB

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)
my_tags=data['profession'].unique().tolist()
# %%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [13]:
# Stocastic Gradient Classifier

# from sklearn.linear_model import SGDClassifier

# sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
#                 #('tfidf', TfidfTransformer()),
#                 ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=50, tol=None)),
#                ])

# a=time()
# sgd.fit(X_train, y_train)

# y_pred_train = sgd.predict(X_train)
# print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
# y_pred = sgd.predict(X_test)
# print('test accuracy %s' % accuracy_score(y_pred, y_test))
    
# b=time()
# print(b-a)
my_tags=data['profession'].unique()

print(classification_report(y_test, y_pred,target_names=my_tags))

                   precision    recall  f1-score   support

          teacher       0.82      0.72      0.77       196
        professor       0.76      0.60      0.67       329
         attorney       0.85      0.90      0.87      1058
          surgeon       0.93      0.59      0.72        73
     photographer       0.82      0.71      0.76       103
          painter       0.77      0.84      0.80       188
     psychologist       0.87      0.94      0.90       432
        filmmaker       0.83      0.80      0.81       137
        physician       0.83      0.72      0.77        47
interior_designer       0.80      0.78      0.79       227
        architect       0.75      0.46      0.57        46
        dietitian       0.68      0.72      0.70       640
           pastor       0.82      0.77      0.80       231
           rapper       0.87      0.77      0.82       589
       journalist       0.75      0.78      0.77       245
          dentist       1.00      0.42      0.59       

In [ ]:
# Random Forest Classifier

rfc = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1))),
                #('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(max_depth=80, verbose=2)),
               ])

a=time()
rfc.fit(X_train, y_train)
y_pred_train = rfc.predict(X_train)
print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
y_pred = rfc.predict(X_test)
print('test accuracy %s' % accuracy_score(y_pred, y_test))
b=time()
print(b-a)

my_tags=data['profession'].unique()
print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
# AdaBoost Classifier

abc = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1))),
                #('tfidf', TfidfTransformer()),
                ('clf', AdaBoostClassifier()),
               ])

a=time()
abc.fit(X_train, y_train)
y_pred_train = abc.predict(X_train)
print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
y_pred = abc.predict(X_test)
print('test accuracy %s' % accuracy_score(y_pred, y_test))
b=time()
print(b-a)

my_tags=data['profession'].unique()
print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
# Logistic Regression
a= time()
from sklearn.linear_model import LogisticRegression
def get_pipe(c):
    logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1))),
                    ('clf', LogisticRegression(n_jobs=1,C=c, verbose=2, max_iter=500)),
                   ])
    return logreg
c=[1,100,1e4,1e6]
for i in c:
    logreg=get_pipe(i)
    logreg.fit(X_train, y_train)
    print('c =', i)
    y_pred_train = abc.predict(X_train)
    print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
    y_pred = abc.predict(X_test)
    print('test accuracy %s' % accuracy_score(y_pred, y_test))
b=time()
print(b-a)

print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
# Gradient Boosting Classifier

gbc = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                #('tfidf', TfidfTransformer()),
                ('clf', GradientBoostingClassifier()),
               ])

a=time()
gbc.fit(X_train, y_train)
y_pred_train = gbc.predict(X_train)
print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
y_pred = gbc.predict(X_test)
print('test accuracy %s' % accuracy_score(y_pred, y_test))
b=time()
print(b-a)

my_tags=data['profession'].unique()
print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
# Linear Support Vector Classifier

def get_pipe(c):
    lsvc = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))),
                    ('clf', LinearSVC(verbose=2, class_weight='balanced', C=c, max_iter=10000)),
                   ])
    return lsvc

my_tags=data['profession'].unique()

a=time()
lis=[0.04]
for i in lis:
    lsvc=get_pipe(i)
    lsvc.fit(X_train, y_train)
    print('c =',i)
    y_pred_train = lsvc.predict(X_train)
    print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
    y_pred = lsvc.predict(X_val)
    print('test accuracy %s' % accuracy_score(y_pred, y_val))
    print(classification_report(y_val, y_pred,target_names=my_tags))
b=time()
print(b-a)


In [ ]:
# Linear Support Vector Classifier with equal split

def get_pipe(c):
    lsvc = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))),
                    ('clf', LinearSVC(verbose=1, class_weight='balanced', C=c, max_iter=10000)),
                   ])
    return lsvc

my_tags=data['profession'].unique()

a=time()
lis=[0.04]
for i in lis:
    lsvc=get_pipe(i)
    lsvc.fit(X_train, y_train)
    print('c =',i)
    y_pred_train = lsvc.predict(X_train)
    print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
    y_pred = lsvc.predict(X_val)
    print('test accuracy %s' % accuracy_score(y_pred, y_val))
    print(classification_report(y_val, y_pred,target_names=my_tags))
b=time()
print(b-a)


In [20]:
# Stocastic Gradient Classifier

from sklearn.linear_model import SGDClassifier

vect=CountVectorizer(ngram_range=(1, 1))
X_vec=vect.fit_transform(X_train)
print(X_vec)

X_csr = csr_matrix(X_vec)
X_dense = X_csr.todense()
print(X_dense)

# sgd = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=50, tol=None)


# a=time()
# sgd.fit(X_dense, y_train)

# y_pred_train = sgd.predict(X_train)
# print('train accuracy %s' % accuracy_score(y_pred_train, y_train))
# y_pred = sgd.predict(X_test)
# print('test accuracy %s' % accuracy_score(y_pred, y_test))
    
# b=time()
# print(b-a)
# my_tags=data['profession'].unique()

# print(classification_report(y_test, y_pred,target_names=my_tags))

  (0, 123155)	1
  (0, 111350)	1
  (0, 20118)	1
  (0, 34490)	1
  (0, 106984)	1
  (0, 141966)	1
  (0, 68065)	2
  (0, 71411)	4
  (0, 34953)	1
  (0, 122074)	1
  (0, 47644)	1
  (0, 40881)	1
  (0, 100744)	1
  (0, 125580)	1
  (0, 45239)	1
  (0, 103990)	1
  (0, 140053)	1
  (0, 122035)	1
  (0, 141521)	1
  (0, 24364)	1
  (0, 60702)	1
  (0, 111725)	1
  (0, 31027)	1
  (0, 48855)	1
  (0, 53238)	1
  :	:
  (103560, 63673)	1
  (103560, 84117)	1
  (103560, 106186)	1
  (103560, 51851)	2
  (103560, 142616)	1
  (103560, 134818)	1
  (103560, 103291)	1
  (103560, 129350)	1
  (103560, 51622)	1
  (103560, 10557)	2
  (103560, 99996)	1
  (103560, 3755)	1
  (103560, 18003)	1
  (103560, 110896)	1
  (103560, 124402)	1
  (103561, 71411)	1
  (103561, 128817)	1
  (103561, 45541)	1
  (103561, 113106)	1
  (103561, 10404)	2
  (103561, 82755)	4
  (103561, 122036)	1
  (103561, 25594)	1
  (103561, 97591)	1
  (103561, 20986)	1


MemoryError: Unable to allocate 112. GiB for an array with shape (103562, 144970) and data type int64